In [8]:
import random
import fasttext


def train_test_split(source_path, train_path, test_path, train_ratio=0.8):
    """划分训练集和测试集"""
    train_file = open(train_path, "w", encoding="utf-8")
    test_file = open(test_path, "w", encoding="utf-8")
    with open(source_path, "r", encoding="utf-8") as f:
        for line in f.readlines():
            if random.random() > train_ratio:
                test_file.write(line)
            else:
                train_file.write(line)
    train_file.close()
    test_file.close()


# 划分训练集和测试集
source_path = "data/news_categorize.txt"
train_path = "data/news_categorize_train.txt"
test_path = "data/news_categorize_test.txt"
train_test_split(source_path, train_path, test_path, train_ratio=0.8)

# 模型训练
model = fasttext.train_supervised(train_path)

# 模型保存与加载
model.save_model("model.bin")
model_load = fasttext.load_model("model.bin")

# 模型评估
test_result = model.test(test_path)
print("测试结果：")
print("样本数：", test_result[0])
print("精确率：", test_result[1])
print("召回率：", test_result[2])

测试结果：
样本数： 76358
精确率： 0.38403310720553185
召回率： 0.38403310720553185


In [13]:
def train_test_split(source_path, train_path, test_path, train_ratio=0.8):
    """划分训练集和测试集"""
    train_file = open(train_path, "w", encoding="utf-8")
    test_file = open(test_path, "w", encoding="utf-8")
    with open(source_path, "r", encoding="utf-8") as f:
        for line in f.readlines():
            label, sentence = line.split(" ", 1)
            sentence = " ".join(list(sentence))  # 将句子拆分为字
            if random.random() > train_ratio:
                test_file.write(label + " " + sentence)
            else:
                train_file.write(label + " " + sentence)
    train_file.close()
    test_file.close()


# 划分训练集和测试集
source_path = "data/news_categorize.txt"
train_path = "data/news_categorize_train.txt"
test_path = "data/news_categorize_test.txt"
train_test_split(source_path, train_path, test_path, train_ratio=0.8)

# 模型训练
model = fasttext.train_supervised(
    input=train_path,
    epoch=10,  # 调整训练轮次
    lr=0.2,  # 调整学习率
    wordNgrams=2,  # 添加n-gram特征
    loss="hs",  # 使用层次softmax，提高训练效率
)

# 模型评估
test_result = model.test(test_path)
print("测试结果：")
print("样本数：", test_result[0])
print("精确率：", test_result[1])
print("召回率：", test_result[2])

测试结果：
样本数： 76262
精确率： 0.8640607379822192
召回率： 0.8640607379822192


In [14]:
# 模型训练
model = fasttext.train_supervised(
    input=train_path,
    epoch=10,  # 调整训练轮次
    lr=0.2,  # 调整学习率，如果学习率太高可能出现NaN
    wordNgrams=2,  # 添加n-gram特征
    loss="hs",  # 使用层次softmax，提高训练效率
    autotuneValidationFile=test_path,  # 使用验证集进行模型调优
    autotuneDuration=300,  # 调优持续时间300秒
    verbose=3,  # 显示详细日志
)

# 模型评估
test_result = model.test(test_path)
print("测试结果：")
print("样本数：", test_result[0])
print("精确率：", test_result[1])
print("召回率：", test_result[2])

测试结果：
样本数： 76262
精确率： 0.8654637958616349
召回率： 0.8654637958616349
